<a href="https://colab.research.google.com/github/besherh/DM-ML/blob/master/committee_of_tflite_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip uninstall -y tensorflow
!pip uninstall -y tf-nightly
!pip install -U -q tf-nightly-gpu


Uninstalling tensorflow-1.14.0:
  Successfully uninstalled tensorflow-1.14.0
     |████████████████████████████████| 228.9MB 48.2MB/s 
     |████████████████████████████████| 4.3MB 48.4MB/s 
     |████████████████████████████████| 512kB 47.7MB/s 
     |████████████████████████████████| 61kB 27.5MB/s 


In [0]:
import tensorflow as tf
tf.enable_eager_execution()
print(tf.__version__)
import numpy as np
import glob




W0808 13:44:47.241978 139674004875136 __init__.py:687] 

  TensorFlow's `tf-nightly` package will soon be updated to TensorFlow 2.0.

  Please upgrade your code to TensorFlow 2.0:
    * https://www.tensorflow.org/beta/guide/migration_guide

  Or install the latest stable TensorFlow 1.X release:
    * `pip install -U "tensorflow==1.*"`

  Otherwise your code may be broken by the change.

  


1.15.0-dev20190808


In [0]:
test_samples = 100

def file_browser(directory_name):
  return [model for model in glob.glob(directory_name + "*.tflite")]

def intialize_test_set():
  (x_train,y_train),(x_test,y_test) = tf.keras.datasets.cifar10.load_data()
  x_test = x_test.astype("float32")
  x_test /= 255
  y_test = tf.keras.utils.to_categorical(y_test,10)
  print(x_test.shape[0], ": testing samples")
  print(y_test.shape, " Lables")
  return x_test,y_test

def model_eval(file_name, x_test, y_test):
  interpreter = tf.lite.Interpreter(file_name)
  interpreter.allocate_tensors()
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]
  model_prediction = []
  for img, label in zip(x_test, y_test):    
    inp = img.reshape((1, 32, 32, 3))
    interpreter.set_tensor(input_index, inp)
    #  start_time = time.time()
    interpreter.invoke()
    #  stop_time = time.time()
    predictions = interpreter.get_tensor(output_index)
    model_prediction.append(predictions)
  return model_prediction

def combine_predictions(model_pool_dic):
  i = 0
  yhats = []
  while i <= test_samples-1:
    one_instance_prediction = []
    for key in model_pool_dic:
      #print("model:",key," item:",i, "is:",model_pool_prediction[key][i])
      one_instance_prediction.append(model_pool_dic[key][i])
    temp = np.array(one_instance_prediction)
    yhat = np.sum(temp, axis=0)
    yhats.append(yhat)  
    i += 1
  return np.array(yhats)

def compute_accuracy(y_test,committee_yhats):
  total_seen = 0
  total_correct = 0
  for i in range(0,test_samples-1):
    total_seen += 1
    #print("Correct label:",np.argmax(y_test[i]), " The predicted label:",np.argmax(committee_yhats[i]))
    if(np.argmax(y_test[i]) == np.argmax(committee_yhats[i])):
      total_correct +=1
  return float(total_correct) / float(total_seen)

 

In [0]:
x_test,y_test = intialize_test_set()

10000 : testing samples
(10000, 10)  Lables


In [0]:
models_list = file_browser("./")
model_pool_prediction = {}
for model_name in models_list:
  model_pool_prediction[model_name] = model_eval(model_name,x_test[:test_samples],y_test[:test_samples])

yhats = combine_predictions(model_pool_prediction)
print(compute_accuracy(y_test,yhats))



0.8282828282828283
